In [94]:
import os 
import telebot 
import datetime as dt 
import jdatetime 
import pymongo


In [95]:
# Get API token from a secure environment variable 
BOT_TOKEN = '7350608336:AAHgBaJR9Vqj2Bl6Mah6XMl_G6G9hPn1gHA' 
 
if not BOT_TOKEN: 
    raise ValueError("Please set the TELEGRAM_BOT_TOKEN environment variable") 
 
bot = telebot.TeleBot(BOT_TOKEN) 

In [96]:
def get_current_persian_datetime(): 
    """Returns the current date and time in Persian format.""" 
    now_persian = jdatetime.datetime.now() 
    formatted_datetime = now_persian.strftime("%H:%M:%S | %A، %d %B %Y") 
    return formatted_datetime 

In [97]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["BidaramDB"]
collection = db["UserInfo"]


In [98]:
@bot.message_handler(commands=['start'])
def handle_start(message):
    chat_id = message.chat.id
    bot.send_message(chat_id, "سلام خوش اومدی ✨")
    
@bot.message_handler(commands=['help'])
def handle_start(message):
    chat_id = message.chat.id
    help_msg = "برای ثبت زمان بیدار شدن از دستور /bidaram استفاده کن"
    bot.send_message(chat_id, help_msg)

@bot.message_handler(commands=['bidaram'])
def handle_bidaram(message):
    chat_id = message.chat.id
    current_datetime_persian = get_current_persian_datetime() 
    today_persian = str(jdatetime.datetime.now()) 
 
    # Get username (handle cases where it might be missing) 
    username = message.from_user.username if message.from_user.username else "Unknown User" 
 
    reply_message = f"⏰{current_datetime_persian}\n\n✨@{username}: {message.text}" 
    username_code = message.from_user.id if message.from_user.id else "0" 
    print_msg =  f"\n@{username}\nusernameCode :{username_code} \n\n {message.text}" 
    print(print_msg) 
    print("⏰ "+  today_persian) 
    
    data = {
    "userName": username,
    "UserID": username_code,
    "Time": today_persian}
    
    try:
        result = collection.insert_one(data)
        print("Data inserted successfully!")
    except pymongo.errors.PyMongoError as e:
        print("Error:", e)
    bot.send_message(chat_id, reply_message)

@bot.message_handler(commands=['list'])
def handle_list(message):
    list_msg = ""
    chat_id = message.chat.id
    username_code = message.from_user.id if message.from_user.id else "0" 
    query = {"UserID": username_code} 
    try:
        cursor = collection.find(query)
        for document in cursor:
            list_msg += str(document["Time"]) + '\n'
    except pymongo.errors.PyMongoError as e:
        print("Error:", e)

    received_text = list_msg
    bot.send_message(chat_id, received_text)

@bot.message_handler(func=lambda message: True)
def handle_all(message):
    chat_id = message.chat.id
    bot.send_message(chat_id, "Sorry, I don't understand that command. Try /help for a list of commands.")

if __name__ == "__main__":
    print('Connection established!')
    bot.infinity_polling()

Connection established!

@oxiiiixo
usernameCode :607284785 

 /bidaram
⏰ 1403-03-25 13:49:06.483964
Data inserted successfully!


In [ ]:
# @bot.message_handler(func=lambda msg: True) 
# def echo_all(message): 
#     """Echos the user's message while displaying the current Persian date and time, and username.""" 
#     current_datetime_persian = get_current_persian_datetime() 
#     today_persian = str(jdatetime.datetime.now()) 
 
#     # Get username (handle cases where it might be missing) 
#     username = message.from_user.username if message.from_user.username else "Unknown User" 
 
#     reply_message = f"⏰{current_datetime_persian}\n\n✨@{username}: {message.text}" 
#     username_code = message.from_user.id if message.from_user.id else "0" 
#     bot.reply_to(message, reply_message) 
#     print_msg =  f"\n@{username}\nusernameCode :{username_code} \n\n {message.text}" 
#     print(print_msg) 
#     print("⏰ "+  today_persian) 
    
#     data = {
#     "userName": username,
#     "UserID": username_code,
#     "Time": today_persian}
    
#     try:
#         result = collection.insert_one(data)
#         print("Data inserted successfully!")
#     except pymongo.errors.PyMongoError as e:
#         print("Error:", e)

 
 
# count = 1 
# while True: 
#     try: 
#         print("Connection established.") 
#         bot.polling(none_stop=True, interval=0)  # Use non-blocking polling for real-time 
#         break  # Successful connection, exit loop 
#     except ConnectionError: 
#         print("Connection failed. Retrying...") 
#     except Exception as e: 
#         print(f"({count}) Unexpected error: {e}. Retrying...") 
#         count += 1  # Track and log errors (optional)